In [10]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
from os.path import join
import torch
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from os.path import join as oj
import torch.utils.data as utils
from torchvision import datasets, transforms
import numpy as np
import os
import sys
pd.set_option('precision', 3)

In [11]:
save_path = "../results_for_export"
trained_model_folder = '../models/MNIST'
fnames = [oj(trained_model_folder, fname) for fname in os.listdir(trained_model_folder)]
# other models were trained badly

results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 
results = pd.concat(results_list, axis=1).T.infer_objects() 
results['final_acc'] = [max(x) for x in results['accs_test']] 

/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [21]:
results['final_acc'] = [max(x) for x in results['accs_test']] 
results['final_acc_train'] = [max(x) for x in results['accs_train']]
results['final_cd'] = [x[-1] for x in results['cd']]
results['final_test_loss'] = [min(x) for x in results['losses_test']]
results['final_train_loss'] = [min(x) for x in results['losses_train']]
results['acc_color'] = [0 for x in results['accs_train']]


# results = results.dropna()

In [22]:
results = results[results.dataset == "Color"]

In [23]:


results.reset_index(drop=True, inplace=True)
results_save = results[['regularizer_rate','final_acc','final_acc_train','final_test_loss', 'acc_color','final_cd', 'method', 'dataset']].sort_values(by = ['regularizer_rate'])

In [24]:
concise_results = results_save.groupby(by = ['dataset', 'method','regularizer_rate']).mean()
concise_results.columns = ['Test accuracy', 'Train accuracy', 'Test loss', 'Color accuracy', 'CD']
# with open(oj(save_path, "color_mnist.text"), 'w') as f:
#           f.write(concise_results.to_latex())

In [25]:
concise_results

Test accuracy  Train accuracy  \
dataset method       regularizer_rate                                  
Color   CDEP         0.0                       0.257         100.000   
                     0.1                       2.883         100.000   
                     1.0                       5.467         100.000   
                     5.0                      13.752          99.935   
                     10.0                     16.693          99.935   
                     50.0                     22.280          99.284   
                     100.0                    22.900          98.242   
                     500.0                    25.117          93.685   
                     1000.0                   23.088          90.755   
                     5000.0                   24.898          99.740   
        ExpectedGrad 0.0                       0.318         100.000   
                     5.0                      11.350          14.062   
                     10.0                     11.350          14.062   
                     50.0                      9.740          20.312   
                     100.0                     9.740          20.312   
        Grad         0.0                       0.266         100.000   
                     1.0                       0.050         100.000   
                     5.0                       0.190         100.000   
                     10.0                      0.248         100.000   
                     50.0                      0.275         100.000   
                     100.0                     0.290         100.000   
                     500.0                     0.327         100.000   
                     1000.0                    0.247         100.000   

                                       Test loss  Color accuracy         CD  
dataset method       regularizer_rate                                        
Color   CDEP         0.0                  31.027               0  0.000e+00  
                     0.1                  26.304               0  6.498e-04  
                     1.0                  18.929               0  7.451e-10  
                     5.0                  13.541               0  1.791e-17  
                     10.0                 11.176               0  5.064e-20  
                     50.0                  6.510               0  2.227e-22  
                     100.0                 5.260               0  3.627e-23  
                     500.0                 3.255               0  1.894e-24  
                     1000.0                2.589               0  6.761e-25  
                     5000.0                7.313               0  1.210e-22  
        ExpectedGrad 0.0                  27.006               0  0.000e+00  
                     5.0                   2.302               0  3.946e-03  
                     10.0                  2.302               0  4.403e-03  
                     50.0                  2.303               0  5.254e-03  
                     100.0                 2.303               0  4.087e-03  
        Grad         0.0                  27.030               0  0.000e+00  
                     1.0                  26.653               0  1.028e-10  
                     5.0                  27.090               0  1.239e-10  
                     10.0                 26.961               0  1.133e-10  
                     50.0                 26.831               0  1.280e-10  
                     100.0                27.024               0  1.393e-10  
                     500.0                27.853               0  1.421e-10  
                     1000.0               27.338               0  1.279e-10

In [26]:
concise_results_for_save = concise_results[['Test accuracy']]#, 'Color accuracy']]

In [27]:
concise_results_for_save

Test accuracy
dataset method       regularizer_rate               
Color   CDEP         0.0                       0.257
                     0.1                       2.883
                     1.0                       5.467
                     5.0                      13.752
                     10.0                     16.693
                     50.0                     22.280
                     100.0                    22.900
                     500.0                    25.117
                     1000.0                   23.088
                     5000.0                   24.898
        ExpectedGrad 0.0                       0.318
                     5.0                      11.350
                     10.0                     11.350
                     50.0                      9.740
                     100.0                     9.740
        Grad         0.0                       0.266
                     1.0                       0.050
                     5.0                       0.190
                     10.0                      0.248
                     50.0                      0.275
                     100.0                     0.290
                     500.0                     0.327
                     1000.0                    0.247

In [28]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_drop = nn.Dropout2d()

        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc1(x))
        
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    def logits(self, x):
    
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc1(x))
        
        x = self.fc2(x)
        return x


In [29]:
def test( model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    return(test_loss, 100.*correct / len(test_loader.dataset))


In [30]:
kwargs = {'num_workers': 1, 'pin_memory': True}
test_x_tensor = torch.Tensor(np.load(oj("../data/ColorMNIST", "test_x.npy")))
test_y_color= torch.Tensor(np.load(oj("../data/ColorMNIST", "test_y.npy"))).type(torch.int64)
test_dataset_color = utils.TensorDataset(test_x_tensor,test_y_color) # create your datset

test_loader_color = utils.DataLoader(test_dataset_color,
        batch_size=256, shuffle=True, **kwargs) # create your dataloader

test_net = Net()
test_net = test_net.to(0)

In [31]:
acc_color_list = []
loss_color_list = []
for i in range(len(results)):
    test_net.load_state_dict(results.model_weights[i])
    loss_col, acc_col = test(test_net, 0, test_loader_color, 0)
    acc_color_list.append(acc_col)
    loss_color_list.append(loss_col)
results["final_acc"] =[x for x in acc_color_list]
results["final_test_loss"] =[x for x in loss_color_list]               
               


In [41]:
kwargs = {'num_workers': 1, 'pin_memory': True}
test_x_tensor = torch.Tensor(np.load(oj("../data/ColorMNIST", "test_x.npy")))
test_y_color= torch.Tensor(np.load(oj("../data/ColorMNIST", "test_y_color.npy"))).type(torch.int64)
test_dataset_color = utils.TensorDataset(test_x_tensor,test_y_color) # create your datset

test_loader_color = utils.DataLoader(test_dataset_color,
        batch_size=256, shuffle=True, **kwargs) # create your dataloader

test_net = Net()
test_net = test_net.to(0)

In [42]:
a = np.load(oj("../data/ColorMNIST", "test_x.npy"))

In [44]:
a[0].shape

(3, 28, 28)

In [39]:
acc_color_list = []
loss_color_list = []
for i in tqdm(range(len(results))):
    test_net.load_state_dict(results.model_weights[i])
    loss_col, acc_col = test(test_net, 0, test_loader_color, 0)
    acc_color_list.append(acc_col)
    loss_color_list.append(loss_col)
results["acc_color"] =[x for x in acc_color_list]
results["loss_color"] =[x for x in loss_color_list]               
               


100%|██████████| 109/109 [00:48<00:00,  2.23it/s]


In [40]:
results[['method', 'regularizer_rate',"final_acc","acc_color",'loss_color']].groupby(by = ['method','regularizer_rate']).mean()

final_acc  acc_color  loss_color
method       regularizer_rate                                  
CDEP         0.0                  13.263     94.480       0.219
             0.1                  14.900     89.077       0.484
             1.0                  16.903     83.320       0.679
             5.0                  20.623     73.737       1.011
             10.0                 22.302     68.787       1.145
             50.0                 26.818     55.482       1.480
             100.0                29.350     49.487       1.606
             500.0                36.713     36.493       1.969
             1000.0               38.643     32.277       2.095
             5000.0               24.898     55.987       1.521
ExpectedGrad 0.0                  13.570     93.860       0.229
             5.0                  11.350      9.480       2.303
             10.0                 11.350      9.480       2.303
             50.0                  9.740      9.920       2.303
             100.0                 9.740      9.920       2.303
Grad         0.0                  13.384     93.867       0.227
             1.0                  12.850     96.550       0.111
             5.0                  13.180     94.142       0.218
             10.0                 13.077     94.482       0.206
             50.0                 13.212     94.027       0.221
             100.0                13.337     94.183       0.214
             500.0                13.392     94.425       0.207
             1000.0               13.283     94.445       0.205